In [ ]:
## THIS CELL SHOULD BE IN ALL VSCODE NOTEBOOKS ##

MARKET = "NSE"

# Set the root
from from_root import from_root

ROOT = from_root()

import pandas as pd
from loguru import logger

pd.options.display.max_columns = None

import sys
from pathlib import Path

# Add `src` and ROOT to _src.pth in .venv to allow imports in VS Code
from sysconfig import get_path

if "src" not in Path.cwd().parts:
    src_path = str(Path(get_path("purelib")) / "_src.pth")
    with open(src_path, "w") as f:
        f.write(str(ROOT / "src\n"))
        f.write(str(ROOT))
        if str(ROOT) not in sys.path:
            sys.path.insert(1, str(ROOT))

logger.add(sink=ROOT / "log" / "ztest.log", mode="w")

# Imports

In [ ]:
import json

import numpy as np
import pandas as pd
import plotly.express as px
import yaml
from mftool import Mftool

In [ ]:
mft = Mftool()

# Fund IDs

In [ ]:
# Get fund IDs of my funds

with open(ROOT / "data" / "mfs.yml", "rb") as f:
    mf_master = yaml.safe_load(f)

descs = {k: v.get("desc") for k, v in mf_master.items()}

ids = {k: next(iter(mft.get_available_schemes(v))) for k, v in descs.items()}

df_mf = pd.DataFrame([descs, ids]).T
df_mf.index.rename("name")
df_mf.columns = ["desc", "id"]

# Historical NAVs

In [ ]:
symbol = "quant_midcap"

ids = [df_mf.loc[s].id for s in descs]


dfs = [
    mft.get_scheme_historical_nav(d, as_Dataframe=True)
    .assign(symbol=symbol)
    .reset_index()
    for d in ids
]

df = pd.concat(dfs, axis=0, ignore_index=True)

df.date = pd.to_datetime(df.date, dayfirst=True)

df.nav = df.nav.astype("float")

# Plot the NAVs